In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# OpenAI APIキーの設定（環境変数から読み取る）
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import pandas as pd

def process_data(df):
    """
    データを処理し、フォーマットされたテキストを返す。
    
    :param df: データフレーム
    :param is_training: トレーニングデータの場合True
    :return: フォーマットされたデータ
    """
    processed_data = []
    for _, row in df.iterrows():
        review = row['review']
        reply = row['replyContent']
        thumbsUpCount = row['thumbsUpCount']
        reviewCreatedVersion = row['reviewCreatedVersion']
        timeToReply = row['timeToReply']
        processed_data.append(f"Review: {review}\nthumbsUpCount: {thumbsUpCount}, reviewCreatedVersion: {reviewCreatedVersion}\nReply: {reply}\ntimeToReply: {timeToReply}")
    return processed_data

# データの読み込み
train_df = pd.read_csv('train_sampled.csv')

new_df = pd.DataFrame()

reviews = process_data(train_df)

scores = train_df["score"]

save_file = "train_formatted.jsonl"

import json

# JSONLファイルに書き込む
with open(save_file, 'w', encoding='utf-8') as f:
    for review, score in zip(reviews, scores):
        data = {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an AI assistant specialized in analyzing detailed app reviews for BANKApp and predicting review scores. Given a review that includes the review text, thumbs up count, app version, official reply , and reply time, you will predict the likely star rating (0-4 stars) the reviewer would give. "
                    #Consider all provided information in your analysis. Factors to consider include the sentiment of the review, the app version, the presence and quality of an official reply, and the response time. Provide only the numerical score without any explanation or additional commentary."
                },
                {
                    "role": "user",
                    "content": review
                },
                {
                    "role": "assistant",
                    "content": str(score)
                }
            ]
        }
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')  # 各JSONオブジェクトの後に改行を追加

print(f"JSONLファイルが作成されました: {save_file}")

from openai import OpenAI

client = OpenAI()

train = client.files.create(
  file=open("train_formatted.jsonl", "rb"),
  purpose="fine-tune"
)
print(train)

valid = client.files.create(
  file=open("validation_formatted.jsonl", "rb"),
  purpose="fine-tune"
)
print(valid)

ft = client.fine_tuning.jobs.create(
  training_file=train.id, 
  validation_file=valid.id,
  model="gpt-3.5-turbo-0125"
)
print(ft)


JSONLファイルが作成されました: train_formatted.jsonl
FileObject(id='file-7CyUaw4esO9MKZcIwtjSDGiz', bytes=5141790, created_at=1723673190, filename='train_formatted.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-axLuUT37rcFoFFO6OfMmvLdK', bytes=319241, created_at=1723673190, filename='validation_formatted.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FineTuningJob(id='ftjob-ZNVFc0ya6uDw5MtIECHb6XQT', created_at=1723673193, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-rso2SGee1kCpNT9F9W8Ek6ak', result_files=[], seed=1491910185, status='validating_files', trained_tokens=None, training_file='file-7CyUaw4esO9MKZcIwtjSDGiz', validation_file='file-axLuUT37rcFoFFO6OfMmvLdK', estimated_finish

In [5]:
from openai import OpenAI
from datetime import datetime

client = OpenAI()

# List 10 fine-tuning jobs
ft_jobs = client.fine_tuning.jobs.list()
for ft_job in ft_jobs:
  id = ft_job.id

  timestamp     = ft_job.created_at
  datetime      = datetime.fromtimestamp(timestamp)
  
  # Retrieve the state of a fine-tune
  state = client.fine_tuning.jobs.retrieve(id).status
  model = client.fine_tuning.jobs.retrieve(id).fine_tuned_model

  if state == 'cancelled':
    continue

  print(f'Create At: {datetime}')
  print(f'FineTune ID: {id}')
  print(f'Model: {model}')
  print(f'Status: {state}\n')


Create At: 2024-08-15 07:06:33
FineTune ID: ftjob-ZNVFc0ya6uDw5MtIECHb6XQT
Model: None
Status: validating_files

Create At: 2024-08-14 22:59:59
FineTune ID: ftjob-dnO21QruCRFc0CugVWczAS56
Model: ft:gpt-4o-mini-2024-07-18:personal::9w9nRfyc
Status: succeeded

Create At: 2024-08-14 21:31:33
FineTune ID: ftjob-NfmBc7D9dm0xIh2Mw7S7ylGC
Model: ft:gpt-4o-mini-2024-07-18:personal::9w87YvQj
Status: succeeded

Create At: 2024-08-14 19:43:36
FineTune ID: ftjob-gh5BqMGuNfhVNscOrCaJDCN5
Model: ft:gpt-4o-mini-2024-07-18:personal::9w6Mw4uL
Status: succeeded

Create At: 2024-08-14 19:14:42
FineTune ID: ftjob-HCZOzt6vxxgcNTEoUepUhKTy
Model: ft:gpt-4o-mini-2024-07-18:personal::9w5vS9JD
Status: succeeded



In [ ]:
!curl https://api.openai.com/v1/fine_tuning/jobs/ftjob-NfmBc7D9dm0xIh2Mw7S7ylGC/checkpoints -H "Authorization: Bearer $OPENAI_API_KEY"

/bin/bash: curl: command not found


In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal::9w5vS9JD", #fine tuningしたモデルのID
  messages=[
    {"role": "system", "content": "You are an AI assistant specialized in analyzing detailed app reviews for BANKApp and predicting review scores. Given a review that includes the review text, thumbs up count, app version, official reply (if any), and reply time, you will predict the likely star rating (0-4 stars) the reviewer would give. Consider all provided information in your analysis. Factors to consider include the sentiment of the review, the app version, the presence and quality of an official reply, and the response time. Provide only the numerical score without any explanation or additional commentary."},
    {"role": "user", "content": "Review: niceay 👌\nthumbsUpCount: 0, reviewCreatedVersion: 33.0\nReply: Hello, BANK friends. We appreciate your feedback; keep boosting your banking transactions with the BANKApp app. Thanks, Rida.\ntimeToReply: 0 days 01::00"},
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='4', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [20]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.retrieve("ftjob-NfmBc7D9dm0xIh2Mw7S7ylGC")

FineTuningJob(id='ftjob-NfmBc7D9dm0xIh2Mw7S7ylGC', created_at=1723638693, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::9w87YvQj', finished_at=1723641266, hyperparameters=Hyperparameters(n_epochs=3, batch_size=2, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-rso2SGee1kCpNT9F9W8Ek6ak', result_files=['file-sEIwuSQrz99DMdwnBd5fDaPY'], seed=1395981742, status='succeeded', trained_tokens=766221, training_file='file-A5lW8c0hP9CHVtgscHg8vP3V', validation_file='file-5MYdnVgcEcyZaOJ7WeexVAEo', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal::9w9nR2PO:ckpt-step-1002",
  messages=[
    {"role": "system", "content": "You are an AI assistant specialized in analyzing detailed app reviews for BANKApp and predicting review scores. Given a review that includes the review text, thumbs up count, app version, official reply , and reply time, you will predict the likely star rating (0-4 stars) the reviewer would give. "}, 
    {"role": "user", "content": "Review: I am unable to withdraw cash through the app because when I first opened the account, I was not provided with an ATM card. Despite the app being updated and the network being stable, the problem persists. What should I do?\nthumbsUpCount: 0, reviewCreatedVersion: 33.0\nReply: Hello BANK Friend, kindly provide all necessary details through Whatsapp 0812 12 14017 or email BANKApp@BANK.co.id for review and follow-up. If everything is in order, we hope to receive 5 stars from your friend. Thanks~Diva\ntimeToReply: 0 days 03:11:00"}
  ]
)
print(completion.choices[0].message)

In [9]:
client.fine_tuning.jobs.cancel("ftjob-ZNVFc0ya6uDw5MtIECHb6XQT")

FineTuningJob(id='ftjob-ZNVFc0ya6uDw5MtIECHb6XQT', created_at=1723673193, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=14, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-rso2SGee1kCpNT9F9W8Ek6ak', result_files=[], seed=1491910185, status='cancelled', trained_tokens=None, training_file='file-7CyUaw4esO9MKZcIwtjSDGiz', validation_file='file-axLuUT37rcFoFFO6OfMmvLdK', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [5]:
from openai import OpenAI
client = OpenAI()

# Delete a fine-tuned model (must be an owner of the org the model was created in)
client.models.delete("ft:gpt-4o-mini-2024-07-18:personal::9w6Mw4uL")

NotFoundError: Error code: 404 - {'error': {'message': "The model 'ft:gpt-4o-mini-2024-07-18:personal::9w6Mw4uL' does not exist", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}